In [1]:
import numpy as np
import os
import time
import pandas as pd
import pickle
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
from typing import Mapping
mp_pose = mp.solutions.pose


In [13]:
!pip install scikit-learn==1.0.1

You should consider upgrading via the 'D:\projects\YogaPoseDetect\yoga_jupy\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
classifier = pickle.load(open('yoga_pose_dataset/latest_model1', 'rb'))

In [5]:
# to calculate angle in three points
def getAngles(item, pt1, pt2, pt3):
    a = item[pt1]
    b = item[pt2]
    c = item[pt3]
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = (np.arccos(cosine_angle))*(180/np.pi)
    return angle

In [3]:
def chk_vis(lis):
    cnt = 0
    for obj in lis:
        if (obj.visibility>0.5): cnt+=1
    if (cnt==33): return True
    else: return False 

In [ ]:

from timeit import default_timer as timer

PosesAudioLoc = "audioFiles/poses/"
SecAudioLoc = "audioFiles/seconds/"


In [36]:
import json
colr = 255
with open("pose_angle.json", "r") as pose_angle:
    pose_data = json.load(pose_angle)
def chck_angle(ps_name, angle_dic):
    angle_col_dic = {}
    for d_angle in pose_data[ps_name]:
        mn, std = pose_data[ps_name][d_angle]
        rat = abs(mn-angle_dic[d_angle])/mn
        if rat>1: rat = 1
        angle_col_dic[d_angle] = (0, colr*(1-rat), colr*rat)
    print(angle_col_dic)
    return angle_col_dic
# print(pose_data)    

In [34]:
def start_vid(frame_rate):
    vid = cv2.VideoCapture(0)
    # frame_rate = 30
    prev = time.time()
    while(True):
        
        # Capture the video frame
        # by frame
        time_elapsed = time.time() - prev
        ret, frame = vid.read()
        # time_elapsed = time.time() - prev
        # ret, frame = self.video.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            with mp_pose.Pose() as pose_tracker:
                result = pose_tracker.process(image=frame)
                # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
                pose_landmarks = result.pose_landmarks
            output_frame = frame.copy()
            # print(pose_landmarks.landmark)
            pose_name = ""
            if pose_landmarks is not None:
                dict_obj = {}
                for i in range(0,33):
                    dict_obj[i] = mp_drawing.DrawingSpec(color=mp_drawing.WHITE_COLOR)
                pose_landmarka = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
                frame_height, frame_width = frame.shape[:2]
                pose_landmarka *= np.array([frame_width, frame_height, frame_width])
                # uncomment this when predicting
                item = pose_landmarka
                angle_dict = {
                    'left_shoulder': getAngles(item, 13, 11, 12),
                    'right_shoulder': getAngles(item, 24, 12, 14),
                    'left_elbow': getAngles(item, 15, 13, 11),
                    'right_elbow': getAngles(item, 12, 14, 16),
                    'left_leg_waist': getAngles(item, 11, 23, 25),
                    'right_leg_waist': getAngles(item, 12, 24, 26),
                    'left_knee': getAngles(item, 23, 25, 27),
                    'right_knee': getAngles(item, 24, 26, 28),
                }
                arr = np.array([angle_dict['left_shoulder'],
                angle_dict['right_shoulder'],
                angle_dict['left_elbow'],
                angle_dict['right_elbow'],
                angle_dict['left_leg_waist'],
                angle_dict['right_leg_waist'],
                angle_dict['left_knee'],
                angle_dict['right_knee']])
                res = classifier.predict([arr]) # classified pose name
                pose_name = res[0]
                angle_colr = chck_angle(pose_name, angle_dict)
                dict_obj[11] = mp_drawing.DrawingSpec(color=angle_colr["left_shoulder"])
                dict_obj[12] = mp_drawing.DrawingSpec(color=angle_colr["right_shoulder"])
                dict_obj[13] = mp_drawing.DrawingSpec(color=angle_colr["left_elbow"])
                dict_obj[14] = mp_drawing.DrawingSpec(color=angle_colr["right_elbow"])
                dict_obj[23] = mp_drawing.DrawingSpec(color=angle_colr["left_leg_waist"])
                dict_obj[24] = mp_drawing.DrawingSpec(color=angle_colr["right_leg_waist"])
                dict_obj[25] = mp_drawing.DrawingSpec(color=angle_colr["left_knee"])
                dict_obj[26] = mp_drawing.DrawingSpec(color=angle_colr["right_knee"])
                x: Mapping[int, mp_drawing.DrawingSpec] = dict_obj
                mp_drawing.draw_landmarks(
                    image=output_frame,
                    landmark_list=pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=  x)
                assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
                # if chk_vis(pose_landmarks.landmark):
                
            output_frame = cv2.putText(output_frame, pose_name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow('frame', output_frame)
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

In [7]:
classifier.classes_

array(['adho mukha savanasana', 'ashtanga namaskar', 'aswa sanchalnasan',
       'bhujangasana', 'pranamasan', 'urdhva hastasana', 'uttanasana'],
      dtype=object)

In [29]:
classifier.metric_params()

2

In [37]:
start_vid(10)

d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 251.99769225335444, 3.002307746645553), 'right_shoulder': (0, 246.00201286957844, 8.997987130421581), 'left_elbow': (0, 252.15683031890327, 2.84316968109671), 'right_elbow': (0, 190.96623321290707, 64.03376678709293), 'left_leg_waist': (0, 156.84652882858225, 98.15347117141773), 'right_leg_waist': (0, 229.27937763083474, 25.720622369165238), 'left_knee': (0, 235.77957124319101, 19.22042875680898), 'right_knee': (0, 180.12399251922085, 74.87600748077915)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 251.8119530416824, 3.1880469583176345), 'right_shoulder': (0, 223.1464445986926, 31.853555401307414), 'left_elbow': (0, 248.16564249652072, 6.834357503479281), 'right_elbow': (0, 196.4797317032596, 58.52026829674039), 'left_leg_waist': (0, 158.296030621919, 96.70396937808101), 'right_leg_waist': (0, 232.4269597491724, 22.57304025082759), 'left_knee': (0, 227.52973090823193, 27.47026909176808), 'right_knee': (0, 179.4450678780977, 75.55493212190231)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


...

{'left_shoulder': (0, 172.571243118121, 82.42875688187897), 'right_shoulder': (0, 24.63701387446219, 230.36298612553782), 'left_elbow': (0, 145.46366449487667, 109.53633550512332), 'right_elbow': (0, 182.84379463410988, 72.15620536589012), 'left_leg_waist': (0, 248.529742947876, 6.470257052123997), 'right_leg_waist': (0, 249.7360902302018, 5.263909769798207), 'left_knee': (0, 203.3090729244755, 51.69092707552449), 'right_knee': (0, 203.02310191972745, 51.97689808027254)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 243.83678728427566, 11.163212715724356), 'right_shoulder': (0, 207.6265333637089, 47.37346663629112), 'left_elbow': (0, 187.63779947243276, 67.36220052756724), 'right_elbow': (0, 243.03047113684343, 11.96952886315656), 'left_leg_waist': (0, 126.09189505558038, 128.90810494441962), 'right_leg_waist': (0, 78.12020573959956, 176.87979426040044), 'left_knee': (0, 252.7635528232164, 2.236447176783596), 'right_knee': (0, 247.82039638268037, 7.179603617319617)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 134.07149689803475, 120.92850310196523), 'right_shoulder': (0, 170.5169603280089, 84.48303967199108), 'left_elbow': (0, 197.08619995950656, 57.91380004049345), 'right_elbow': (0, 191.9481088596303, 63.0518911403697), 'left_leg_waist': (0, 231.62127443634103, 23.378725563658982), 'right_leg_waist': (0, 205.75670773568086, 49.24329226431916), 'left_knee': (0, 220.937509457326, 34.062490542674), 'right_knee': (0, 210.96287459870345, 44.03712540129654)}
